In [45]:
# https://stackoverflow.com/questions/1456373/two-way-reverse-map
class TwoWayDict(dict):
    def __setitem__(self, key, value):
        if key in self:
            del self[key]
        if value in self:
            del self[value]
        dict.__setitem__(self, key, value)
        dict.__setitem__(self, value, key)
    def __len__(self):
        """Returns the number of connections"""
        return dict.__len__(self) // 2

In [46]:
def abs_distance(string: str, frequencies: {str: float}) -> float:
    
    return sum([
        abs(frequencies[c] - string.count(c) / len(string))
        for c in frequencies
    ])

In [78]:
def shift_word(word: str, index: int, characters_index: TwoWayDict) -> str:
    shifted_chars = []
    
    for char in word:
        new_index = (index + characters_index[char]) % len(characters_index)
        new_char = characters_index[new_index]
        shifted_chars.append(new_char)

    return ''.join(shifted_chars)

In [89]:
def find_best_character_shift(
    encrypted_string: str,
    characters: list[str],
    characters_index: TwoWayDict,
    distance: Callable[[str, {str: float}], float],
    frequencies: {str: float},
) -> (str, float):
    best_character = characters[0]
    min_distance = len(encrypted_string)

    for char in characters:
        index = characters_index[char]
        decrypted_string = shift_word(encrypted_string, index, characters_index)
        char_distance = distance(decrypted_string, frequencies)
        if char_distance < min_distance:
            best_character = char
            min_distance = char_distance
    
    return best_character, min_distance

In [101]:
from collections.abc import Callable

def break_rp(
    ciphertext: str,
    frequencies: {str: float},
    distance: Callable[[str, {str: float}], float],
    ) -> str:
    
    characters = sorted(frequencies.keys())
    characters_index = TwoWayDict()
    for i, char in enumerate(characters):  
        characters_index[i] = char
    
    max_key_length = len(ciphertext) // 50
    
    best_distance = len(ciphertext)
    best_key = None
    for key_length in range(1, max_key_length + 1):
        key_list = []
        total_distance = 0
        for position in range(key_length):
            encrypted_substring = ciphertext[position::key_length]
            best_character, min_distance = find_best_character_shift(
                encrypted_substring,
                characters,
                characters_index,
                distance,
                frequencies,
            )

            key_list.append(best_character)
            total_distance += min_distance

        print('key length', key_length, total_distance)
        if total_distance < best_distance:
            best_distance = total_distance
            best_key = ''.join(key_list)
    
    print(best_key)
    print(best_distance)
    
    return best_key

In [103]:
break_rp('ssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssssss',
         {'a': 0.8, 's': 0.05, 'd': 0.1, 'f': 0.05}, abs_distance)

key length 1 0.39999999999999997
key length 2 0.7999999999999999
d
0.39999999999999997


'd'

In [79]:

characters = ['a', 'b']
characters_index = TwoWayDict()
characters_index[0] = 'a'
characters_index[1] = 'b'
shift_word('aaaa', 1, characters_index)

'bbbb'